In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn 
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import random_split, Dataset, DataLoader
import random 
from tqdm import tqdm 
import warnings
import sys
import os
from PIL import Image

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp -r '/content/drive/MyDrive/trainval' '/content/trainval'


In [4]:
!cp -r '/content/drive/MyDrive/test' '/content/test'

In [5]:
device = torch.cuda.is_available()
if device:  
  print('CUDA is found! Tranining on %s.......'%torch.cuda.get_device_name(0))
else:
  warnings.warn('CUDA not found! Training may be slow......')

CUDA is found! Tranining on Tesla T4.......


In [6]:
from glob import glob
 
transform_train = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.Resize((224)),
    transforms.CenterCrop((224,224)),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])       


transform_test = transforms.Compose([
    transforms.Resize((300,600)),   
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
])
 
 
class MyDataset_train(Dataset):
    def __init__(self, img_path, transform=None):
        super(MyDataset_train, self).__init__()
        self.root = img_path
 
        self.txt_root = self.root + '/' + 'trainval_labels.csv'
 
        f = open(self.txt_root, 'r')
        data = f.readlines()
 
        imgs = []
        labels = []
        for line in data[1:]:
            line = line.rstrip()
            word = line.split(",")
            imgs.append(word[0])
 
            labels.append(word[1])
        self.img = imgs
        self.label = labels
        self.transform = transform
 
    def __len__(self):
        return len(self.label)
 
    def __getitem__(self, item):
        img = self.img[item]
        label = self.label[item]
 
        img = Image.open(img).convert('RGB')
 
        img = self.transform(img)
 
        label = np.array(label).astype(np.int64)
        label = torch.from_numpy(label)
 
        return img, label

class MyDataset_test(Dataset):
    def __init__(self, img_path, transform=None):
        super(MyDataset_test, self).__init__()
        self.root = img_path
 
        self.txt_root = self.root + '/' + 'test_labels.csv'
 
        f = open(self.txt_root, 'r')
        data = f.readlines()
 
        imgs = []
        for line in data[1:]:
            imgs.append(line.strip())
 
        self.img = imgs

        self.transform = transform
 
    def __len__(self):
        return len(self.img)
 
    def __getitem__(self, item):
        img = self.img[item]
        img_name = img.split('/')[-2] + '/' + img.split('/')[-1].replace('_image.jpg', '')
        img = Image.open(img).convert('RGB')
 
        img = self.transform(img)
 
        return img, img_name

In [7]:
path_train = '/content/trainval'

dataset = MyDataset_train(path_train, transform=transform_train)

# val_size = int(0.1 * len(dataset))
# train_size = len(dataset) - val_size
# train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

fast_size = int(0.1 * len(dataset))
fast_drop_size = len(dataset) - fast_size
fast_dataset, fast_drop_dataset = torch.utils.data.random_split(dataset, [fast_size, fast_drop_size])

val_size = int(0.02 * len(dataset))
train_size = fast_size - val_size
train_dataset, val_dataset = torch.utils.data.random_split(fast_dataset, [train_size, val_size])


train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

path_test = '/content/test'

test_dataset = MyDataset_test(path_test, transform=transform_test)
test_loader= DataLoader(test_dataset, batch_size=1, shuffle=False)

In [8]:
print("Total Train Samples: ", len(train_dataset))
print("Total Validation Samples: ", len(val_dataset))
print("Total Test Samples: ", len(test_dataset))

Total Train Samples:  606
Total Validation Samples:  151
Total Test Samples:  2631


In [9]:
def add(in_features, hidden_layers, out_features):
    add = nn.Sequential()
    layer_sizes = zip(hidden_layers[:-1], hidden_layers[1:])
    add.add_module('fc0', nn.Linear(in_features, hidden_layers[0]))
    add.add_module('relu0', nn.ReLU())
    add.add_module('drop0', nn.Dropout(0.6))
    
    for i, (h1, h2) in enumerate(layer_sizes):
        add.add_module('fc'+str(i+1), nn.Linear(h1, h2))
        add.add_module('relu'+str(i+1), nn.ReLU())
        add.add_module('drop'+str(i+1), nn.Dropout(0.5))
        
    add.add_module('output', nn.Linear(hidden_layers[-1], out_features))
        
    return add

In [16]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = models.resnet18(pretrained=True)

for parameter in model.parameters():
    parameter.require_grad = False

added = add(512, [1000], 3)
model.fc = added

checkpoint = torch.load('model_restnet18_fast.pth')
model.load_state_dict(checkpoint)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

model.to(device)

num_epochs = 10

In [14]:
torch.manual_seed(0)
np.random.seed(0)
torch.cuda.manual_seed(0)

count = 0
loss_list = []
iteration_list = []
accuracy_list = []

for epoch in tqdm(range(num_epochs)):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        
        outputs = model(images)
        pred = torch.argmax(outputs, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        count += 1
    
        if count % 50 == 0:
            model.eval()         
            correct = 0
            total = 0

            pred = []
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)                
                outputs = model(images)               
                predicted = torch.argmax(outputs, 1)
                pred.append(predicted)
                total += len(labels)       
                correct += (predicted == labels).sum()
            
            # print(pred)
            accuracy = 100 * correct / float(total)            
            loss_list.append(loss.item())
            iteration_list.append(count)
            accuracy_list.append(accuracy.item())
        if count % 50 == 0:
            print('Iteration: {}  Loss: {}  Accuracy: {} %'.format(count, loss.data, accuracy))

torch.save(model.state_dict(), 'model.pth')

 10%|█         | 1/10 [00:38<05:46, 38.52s/it]

Iteration: 50  Loss: 0.7080507874488831  Accuracy: 52.3178825378418 %


 20%|██        | 2/10 [01:25<05:47, 43.43s/it]

Iteration: 100  Loss: 1.011859655380249  Accuracy: 62.91390609741211 %


 30%|███       | 3/10 [02:14<05:21, 45.98s/it]

...

 70%|███████   | 7/10 [05:14<02:17, 45.85s/it]

Iteration: 300  Loss: 0.5297276377677917  Accuracy: 63.5761604309082 %


 90%|█████████ | 9/10 [06:40<00:43, 43.79s/it]

Iteration: 350  Loss: 0.6990066766738892  Accuracy: 64.23841094970703 %


100%|██████████| 10/10 [07:27<00:00, 44.73s/it]


In [ ]:
# import shutil

# source_file_path = "/content/model.pth"
# destination_folder_path = "/content/drive/MyDrive"
# shutil.copy(source_file_path, destination_folder_path)

In [18]:
import csv

predicted_label = []

model.eval()
name = 'results.csv'
with open(name, 'w') as f:
    writer = csv.writer(f, delimiter=',', lineterminator='\n')
    writer.writerow(['guid/image', 'label'])
    for images, name in test_loader:
        images = images.to(device)
        outputs = model(images)
        predicted = torch.argmax(outputs, 1)
        predicted_label.append(predicted)
        name = str(name)
        name = name.replace('(', '')
        name = name.replace(')', '')
        name = name.replace(',', '')
        name = name.replace('\'', '')
        writer.writerow([name, str(predicted.item())])